# ДЗ №3

Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.

## Библиотеки и загрузка данных

In [83]:
import re
import numpy as np
import pandas as pd
import spacy
import nltk
import en_core_web_md

from collections import Counter

In [80]:
df = pd.read_pickle('../lesson01/result.pkl.zip', compression='zip')
df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguid, societi, motiv]"


In [108]:
train_clean_tweet = ' '.join(df.clean_tweet.values)[:1000000]
test_clean_tweet = ' '.join(df[df['label'].isnull()].clean_tweet.values)[:1000000]

## Задание 1.

Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. 

Действительно ли в топ вошли только персоны и организации или есть мусор?

In [109]:
nlp = en_core_web_md.load()

In [110]:
def getNERCounters(text, nlp):
    cp = Counter()
    cn = Counter()
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ not in ['DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']:
            cp[ent.label_] += 1
        if ent.label_ in ['PERSON', 'ORG']:
            cn[ent.text] += 1
    return cp.most_common(3), cn.most_common(20)

In [111]:
train_counters = getNERCounters(train_clean_tweet, nlp)

In [112]:
print('для TRAIN данных')
print('   TOP 3 типов: ', train_counters[0])
print('   TOP 20 персон/организаций: ', train_counters[1])

для TRAIN данных
   TOP 3 типов:  [('PERSON', 3374), ('GPE', 1671), ('ORG', 1473)]
   TOP 20 персон/организаций:  [('obama', 28), ('sta', 24), ('deletetweets', 20), ('suppo', 20), ('islam', 20), ('nba', 19), ('gop', 19), ('hillary', 18), ('sjw', 16), ('bing bong bing bong', 14), ('lebron', 11), ('bjp', 11), ('christina grimmie', 10), ('shi', 10), ('melancholy melancholymusic', 10), ('fed', 10), ('wso', 10), ('bihday', 10), ('cavs', 10), ('hu', 9)]


In [114]:
test_counters = getNERCounters(test_clean_tweet, nlp)

In [115]:
print('для TEST данных')
print('   TOP 3 типов: ', test_counters[0])
print('   TOP 20 персон/организаций: ', test_counters[1])

для TEST данных
   TOP 3 типов:  [('PERSON', 3392), ('GPE', 1535), ('ORG', 1484)]
   TOP 20 персон/организаций:  [('suppo', 28), ('obama', 26), ('sta', 21), ('gop', 20), ('ht', 16), ('nba', 16), ('deletetweets', 14), ('christina grimmie', 12), ('islam', 11), ('feminismisterrorism feminismmuktbharat', 11), ('sjw', 11), ('clinton', 10), ('wa', 10), ('bihday', 9), ('facebook', 9), ('hu', 9), ('donald trump', 9), ('lebron', 9), ('jo cox', 9), ('bong bing bong', 9)]


> **Вывод**

> * Для тестовых и обучаемых данных самый популярный тип PERSON 
>   * на первом месте в TRAIN - Omaba
>   * на первом месте в TEST - Suppo - скорее всего мусор ? на втором месте Obama
> * В результате есть мусор


## Задание 2.

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. 

Действительно ли в топ вошли только персоны и организации или есть мусор?

In [212]:
train_clean_tweet = ' '.join(df.tweet.values)[:1000000]
test_clean_tweet = ' '.join(df[df['label'].isnull()].tweet.values)[:1000000]

In [213]:
def getNLTKCounters(text):
    cp = Counter()
    cn = Counter()
    chunks = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))
    for chunk in chunks:
        if hasattr(chunk, 'label'):
            txt = ' '.join(c[0] for c in chunk)
            label = chunk.label()
            if label not in ['DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']:
                cp[label] += 1
            if ent.label_ in ['PERSON', 'ORGANIZATION']:
                cn[txt] += 1
    return cp.most_common(3), cn.most_common(20)

In [214]:
train_counters = getNLTKCounters(train_clean_tweet)

In [215]:
print('для TRAIN данных')
print('   TOP 3 типов: ', train_counters[0])
print('   TOP 20 персон/организаций: ', train_counters[1])

для TRAIN данных
   TOP 3 типов:  [('ORGANIZATION', 13), ('GPE', 5), ('PERSON', 1)]
   TOP 20 персон/организаций:  [('u.s.', 3), ('en-ger-land', 1), ('nycÂ', 1), ('travelÂ', 1), ('mr. velicaria', 1), ('atÂ', 1), ('whiteaddictÂ', 1), ('inc.', 1), ('leastâ\x80¦', 1), ('perÃº', 1), ('sÃ³ria', 1), ('____________________________________', 1), ('firstÂ', 1), ('moanaÂ', 1), ('ludik.hugo', 1), ('isrâ\x80¦', 1), ('littleÂ', 1)]


In [216]:
test_counters = getNLTKCounters(test_clean_tweet)

In [217]:
print('для TEST данных')
print('   TOP 3 типов: ', test_counters[0])
print('   TOP 20 персон/организаций: ', test_counters[1])

для TEST данных
   TOP 3 типов:  [('ORGANIZATION', 19), ('GPE', 9), ('PERSON', 2)]
   TOP 20 персон/организаций:  [('u.s.', 7), ('fÃªte', 2), ('mr.', 2), ('whatÂ', 1), ('ó¾\x93¯', 1), ('felicitÃ', 1), ('aldilÃ', 1), ('___', 1), ('yoursÂ', 1), ('peace/sad', 1), ('________________________________________', 1), ('newsÂ', 1), ('coolÂ', 1), ('liberal/left', 1), ('husbandÂ', 1), ('atÂ', 1), ('st.', 1), ('zÃ¼rich', 1), ('gÃ¼mbet', 1), ('u.s', 1)]


> **Вывод**: нужно брать не очищенные данные и даже в этом случае много мусора потому что если делать lower вообще ничего не находит

## Задание 3.

Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.

> **Лучше всего отработала Spacy потому, для NLTK пришлось делать какой то хак потому что она не понимала lower() кроме того NLTK работает очень медлено и находит меньше NER а больше мусора**